In [1]:
import xml.etree.ElementTree as ET
import xmljson
import pandas as pd
import pyodbc
import os
from pprint import pprint
import xmltodict

pd.set_option('max_colwidth', 160)
pd.set_option('mode.chained_assignment', None)

conn_str = r"Driver={Microsoft Access Driver (*.mdb, *.accdb)};"\
    r"DBQ=C:\Users\gdoku\YandexDisk\Документы\Семейные финансы\FF.accdb;"


def vtb_broker_security_deals(path):
    """Считываем данные по движению денежных средств из брокерского отчёта Банка ВТБ в формате xml"""

    directory = r'C:/Users/gdoku/YandexDisk/Документы/Договоры/Банк ВТБ (ПАО)/'
    files = os.listdir(os.path.join(directory, path))
    counter = 0

    # Считываем данные из файлов
    for file in files:
        tree = ET.parse(os.path.join(directory, path, file))
        root = tree.getroot()
        json = xmljson.gdata.data(root[17][0])
        json = json['{report577p_v1}Подробности9_Collection']['{report577p_v1}Подробности9']
        data = pd.DataFrame(json)
        if counter == 0:
            security_deals = data
            counter += 1
        else:
            security_deals = transactions.append(data, ignore_index=True)

    # Обработка данных
    security_deals = security_deals[['NameBeg9', 'curs_datebeg9', 'currency_ISO9',
                                     'NameEnd9', 'currency_paym7', 'deal_cost7',
                                     'bank_сommition7', 'deal_code5', 'deal_code1']]
    security_deals.columns = ['security', 'date', 'deal_type', 'quantity',
                              'amount', 'accumulated_coupon', 'deal_сommission', 'bank_сommission', 'deal_code']
    security_deals.date = pd.to_datetime(
        security_deals.loc[:, 'date'], format='%Y-%m-%d').dt.normalize()
    security_deals.security = security_deals.security.apply(lambda x: x[-12:])
    security_deals.quantity = security_deals.apply(
        lambda x: x.quantity if x.deal_type == 'Покупка' else -x.quantity, axis=1)
    security_deals.amount = security_deals.apply(
        lambda x: x.amount if x.deal_type == 'Покупка' else -x.amount, axis=1)
    security_deals.accumulated_coupon = security_deals.apply(
        lambda x: x.accumulated_coupon if x.deal_type == 'Покупка' else -x.accumulated_coupon, axis=1)
    security_deals['сommission'] = security_deals.deal_сommission + security_deals.bank_сommission
    security_deals = security_deals.iloc[:,[0,1,3,4,5,8,9]]

    return security_deals

path = r'Георгий/Брокерские отчёты XML'
security_deals = vtb_broker_security_deals(path)
security_deals

,security,date,quantity,amount,accumulated_coupon,deal_code,сommission
0,RU0007661625,2020-02-26,80.0,17824.80,0.00,B3103078226,8.51
1,RU0007976965,2020-02-26,5.0,8775.00,0.00,B3103079669,4.18
2,RU0007976965,2020-02-26,6.0,10548.00,0.00,B3103089622,5.04
3,RU0007661625,2020-02-27,80.0,17679.20,0.00,B3103796348,8.45
4,RU0007976965,2020-02-27,5.0,8752.50,0.00,B3103796983,4.17
...,...,...,...,...,...,...,...
345,RU000A101FT1,2020-07-27,30.0,26597.40,782.40,B3245547587,12.33
346,RU000A0JPFP0,2020-07-28,26.0,19266.00,0.00,B3247642677,9.20
347,RU000A0JPFP0,2020-07-29,25.0,18510.00,0.00,B3248053926,8.84
348,RU000A0JPFP0,2020-07-30,10.0,7360.00,0.00,B3249817538,3.51


In [5]:
def new_security_deals(security_deals):
    """Подготовка таблицы для загрузки."""

    start_date = '2020-01-01'
    conn = pyodbc.connect(conn_str)
    SQL = 'SELECT DISTINCT [003_Факты].transaction_code \
            FROM 003_Факты INNER JOIN \
            004_Проводки ON [003_Факты].ФактНом = [004_Проводки].ФактНом \
            WHERE ((([003_Факты].transaction_code) Is Not Null) AND \
                    ([004_Проводки].СинтСчётНом=581 Or [004_Проводки].СинтСчётНом=582))'
    base = pd.read_sql(SQL, conn)
    security_deals = security_deals[~security_deals.deal_code.isin(base['transaction_code'])]
    security_deals = security_deals[security_deals.date >= start_date]

    SQL = 'SELECT СинтСчётНом AS syn_acc, АналСчётНом AS anal_acc, acc_code \
            FROM 002_АналСчета \
            WHERE (СинтСчётНом=581 Or СинтСчётНом=582) AND acc_code Is Not Null'
    mapping = pd.read_sql(SQL, conn)
    security_deals = pd.merge(security_deals, mapping, how='left',
                              left_on=['security'],
                              right_on=['acc_code']).iloc[:, [0, 1, 2, 3, 4, 5, 6, 7, 8]]
    conn.close()

    return security_deals


def security_deals_upload_to_base(security_deals):
    """Функция загрузки новых проводок в базу."""

    security_deals = new_security_deals(security_deals)
    conn = pyodbc.connect(conn_str)
    cur = conn.cursor()

    # Проведение проводок
    post = 'Купля-продажа ЦБ'
    for i, row in security_deals.iterrows():
        k = 0
        cur.execute('INSERT INTO 003_Факты ( Дата, СобНаим, transaction_code ) VALUES(?, ?, ?)',
                    row['date'], post, row['deal_code'])
        post_id = cur.execute('SELECT Max(ФактНом) FROM 003_Факты WHERE СобНаим = ?',
                              post).fetchall()[0][0]
        cur.execute('INSERT INTO 004_Проводки VALUES(?, ?, ?, ?, ?, ?)',
                    post_id, k, row['syn_acc'], row['anal_acc'],
                    row['amount']-row['accumulated_coupon']+row['сommission'], row['quantity'])
        if row['syn_acc'] == 582:
            k += 1
            cur.execute('INSERT INTO 004_Проводки VALUES(?, ?, ?, ?, ?, ?)',
                        post_id, k, row['syn_acc'], row['anal_acc'],
                        row['accumulated_coupon'], 0)
        k += 1
        cur.execute('INSERT INTO 004_Проводки VALUES(?, ?, ?, ?, ?, ?)',
                    post_id, k, 760, 53, -row['amount'], 0)
        k += 1
        cur.execute('INSERT INTO 004_Проводки VALUES(?, ?, ?, ?, ?, ?)',
                    post_id, k, 760, 57, -row['сommission'], 0)
        if row['syn_acc'] == 581:
            k += 1
            cur.execute('INSERT INTO 004_Проводки VALUES(?, ?, ?, ?, ?, ?)',
                        post_id, k, 792, 19, -row['amount']-row['сommission'], 0)
            k += 1
            cur.execute('INSERT INTO 004_Проводки VALUES(?, ?, ?, ?, ?, ?)',
                        post_id, k, 792, 20, row['amount']+row['сommission'], 0)
    conn.commit()
    conn.close()
    print('Добавлено операций купли-продажи ценных бумаг: {}'.format(len(security_deals)))


# df = new_security_deals(security_deals)
# df
security_deals_upload_to_base(security_deals)

Добавлено операций купли-продажи ценных бумаг: 350


In [9]:
path = r'Георгий/Брокерские отчёты XML'
directory = r'C:/Users/gdoku/YandexDisk/Документы/Договоры/Банк ВТБ (ПАО)/'
files = os.listdir(os.path.join(directory, path))
files = [os.path.join(directory, path, file) for file in files]
files

['C:/Users/gdoku/YandexDisk/Документы/Договоры/Банк ВТБ (ПАО)/Георгий/Брокерские отчёты XML\\72329080_20200101_20200917_381428.xml']

In [10]:
max(files, key=os.path.getctime)

'C:/Users/gdoku/YandexDisk/Документы/Договоры/Банк ВТБ (ПАО)/Георгий/Брокерские отчёты XML\\72329080_20200101_20200917_381428.xml'

In [ ]:
os.path.getctime

In [107]:
security_deals[security_deals.date < '2020-04-01']

,security,date,quantity,amount,accumulated_coupon,deal_code,сommission
0,RU0007661625,2020-02-26,80.0,17824.8,0.0,B3103078226,8.51
1,RU0007976965,2020-02-26,5.0,8775.0,0.0,B3103079669,4.18
2,RU0007976965,2020-02-26,6.0,10548.0,0.0,B3103089622,5.04
3,RU0007661625,2020-02-27,80.0,17679.2,0.0,B3103796348,8.45
4,RU0007976965,2020-02-27,5.0,8752.5,0.0,B3103796983,4.17
...,...,...,...,...,...,...,...
249,RU000A0JPV70,2020-03-27,-160.0,-9480.0,-0.0,S3143741939,4.53
250,RU000A0JL4R1,2020-03-27,-1.0,-1817.5,-0.0,S3143911650,0.87
251,RU000A0JL4R1,2020-03-27,-3.0,-5452.5,-0.0,S3143916504,2.60
252,RU000A0JL4R1,2020-03-27,-9.0,-16357.5,-0.0,S3144126084,7.82


In [4]:
conn = pyodbc.connect(conn_str)